In [ ]:
adset_payload = {
    "name": "Genesis AI - Target Entrepreneurs (25-45)",  # Human-readable name for the Ad Set
    "campaign_id": campaign_id,  # The ID returned when the campaign was created
    "daily_budget": 2000,  # Budget in *cents* (i.e. $20/day = 2000)
    "billing_event": "IMPRESSIONS",  # What you're billed for; common values: IMPRESSIONS, CLICKS
    "optimization_goal": "LINK_CLICKS",  # What the ad will optimize for (clicks, conversions, etc.)
    "bid_strategy": "LOWEST_COST_WITHOUT_CAP",  # Let Meta optimize bids automatically
    "status": "PAUSED",  # Initial status; start paused so you can review
    "start_time": "2025-06-24T09:00:00-0700",  # Optional start time (ISO 8601 format)
    "end_time": "2025-07-01T23:59:00-0700",  # Optional end time
    "targeting": {
        # Minimum age of targeted audience
        "age_min": 25,
        # Maximum age of targeted audience
        "age_max": 45,
        # Gender targeting: 1 = male, 2 = female, [1,2] = all
        "genders": [1, 2],

        # Target by geography
        "geo_locations": {
            "countries": ["US", "CA"],  # You can also use regions or cities
            "location_types": ["home", "recent"]
            # home = people who live in the location
            # recent = people who were recently in the location
        },

        # Interest-based targeting
        "interests": [
            {"id": "6003139266461", "name": "Business"},
            {"id": "6003349442621", "name": "Entrepreneurship"},
            {"id": "6003277229372", "name": "Technology"}
        ],
        # Get these interest IDs from a smart agent (explained below)

        # Ad placement: where the ads will appear
        "publisher_platforms": ["facebook", "instagram"],
        "device_platforms": ["mobile", "desktop"],

        # Optional: combine audience traits using AND logic (not just OR like interests above)
        "flexible_spec": [
            {"interests": [{"id": "6003139266461", "name": "Business"}]},
            {"interests": [{"id": "6003349442621", "name": "Entrepreneurship"}]}
        ],

        # Optional: Smart automation by Meta (Advantage+ options)
        "targeting_automation": {
            "advantage_audience": 0,  # 1 to let Meta expand audience beyond targeting
            "individual_setting": {
                "age": 1,  # Meta can optimize age range
                "gender": 1  # Meta can optimize gender
            }
        }
    }
}


In [3]:
payload = {
'campaign_name': 'Home Workout routine - Instagram',
'campaign_id': 8,
'headline': 'Get Fit at Home - No Gym Needed!',
'platform': 'Instagram',
'objective': 'conversions',
'ad_type': 'Video',
'budget': '20.00',
 'audience': {
             'interests': ['Fitness', 'Home workouts', 'Health and wellness', 'Yoga', 'Bodyweight training'], 
              'behaviors': ['Frequent gym-goers looking for alternatives', 'Busy professionals seeking fitness solutions', 'Health-conscious individuals'],
              'demographics': {'age_min': 20, 'age_max': 40,
              'education_statuses': ['High School', 'College grads'],
                'relationship_statuses': ['Single', 'Married', 'In a relationship']},
              'gender': 'all'
             },
'creatives': [{'type': 'Video', 'url': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg',
'headline': 'Get Fit at Home - No Gym Needed!',
'cta': 'Get Started', 'ad_copy': 'Transform your home into a personal gym with our Ultimate Home Workout Routine. No equipment required, just your determination!'}],
'creatives_attached': 1,
'start_date': datetime.datetime(2025, 6, 22, 17, 9, 33, 636599, tzinfo=datetime.timezone.utc),
'end_time': None

}

NameError: name 'datetime' is not defined

In [ ]:
"targeting": { 
  "age_range": [25, 35], 
  "geo_locations": { 
  	"countries": ["GB"],
  	"cities": [{"key":"2430536", "radius":12, "distance_unit":"mile"}]
  },
  "targeting_automation": { 
  	"individual_setting": {
  		"geo": 1
  	} 
  } 
}

In [4]:
def search_interests(query: str):
    url = "https://graph.facebook.com/v19.0/search"
    params = {
        "q": query,
        "type": "adinterest",
        "limit": 25,
        "access_token": os.getenv("META_SYSTEM_USER_TOKEN")
    }
    response = requests.get(url, params=params)
    return response.json()


In [ ]:
 [
{'id': '6003225004145', 'name': 'Project management (business and finance)'},
{'id': '6003371567474', 'name': 'Entrepreneurship (business and finance)'},
{'id': '6003374632277','name': 'Freelancer (careers)'}
 ]
[{'id': '6003225004145', 'name': 'Project management (business and finance)'},
 {'id': '6003371567474', 'name': 'Entrepreneurship (business and finance)'},
 {'id': '6003374632277', 'name': 'Freelancer (careers)'}]

In [ ]:
metrics = {
    "campaign_id": "127",
    "ctr": 0.9,
    "purchase_roas": 2.0,
    "cpc": 0.7,
    "spend": 60,
    "purchases": 3,
    "frequency": 2.0
}


In [ ]:
def clone_campaign(meta_campaign_id: str) -> Dict:
    """
    Clones a Meta campaign intelligently using deep copy with suffix timestamp.
    Optionally applies variations based on decision data.

    Args:
        campaign_id (str): Original Meta campaign ID
        decision_data (dict): Optional decision data (for context or variation logic)

    Returns:
        dict: Result of cloning operation
    """
    try:
        # 🎯 Create a unique suffix to identify the clone
        suffix = f"Clone_{datetime.now().strftime('%m%d')}"

        # 🔁 Clone campaign via API
        url = f"{BASE_URL}/{meta_campaign_id}/copies"
        payload = {
            "access_token": ACCESS_TOKEN,
            "deep_copy": True,
            "rename_options": {
                "rename_strategy": "APPEND_SUFFIX",
                "rename_suffix": suffix
            },
            "status_option": "INHERITED"  # Start with same status as original
        }

        response = requests.post(url, json=payload)
        response.raise_for_status()
        clone_data = response.json()

        # ✅ Extract new campaign ID from Meta response
        new_campaign_id = clone_data.get("copied_campaign_id")

        # 🛠 Optional: Apply variations (creative, audience, etc.)
        variation_result = apply_clone_variations(new_campaign_id, decision_data)

        return {
            "status": "success",
            "action": "clone",
            "original_campaign_id": campaign_id,
            "new_campaign_id": new_campaign_id,
            "variations_applied": variation_result,
            "response": clone_data
        }

    except Exception as e:
        return {
            "status": "failed",
            "action": "clone",
            "campaign_id": campaign_id,
            "error": str(e),
            "fallback": "Manual campaign duplication recommended"
        }
